<a href="https://colab.research.google.com/github/choeuneheol/python-practice/blob/master/%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0%EC%B1%85DAY6(%EB%B2%94%EC%A3%BC%ED%98%95_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#베이스라인 모델
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
suvimission = pd.read_csv('sample_submission.csv')

In [3]:
all_data = pd.concat([train, test])         # 훈련 데이터와 테스트 데이터 합치기
all_data = all_data.drop('target', axis=1)  # 타깃값 제거
all_data

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,499995,0,0,0,F,N,Green,Square,Lion,Canada,...,9e4b23160,acc31291f,1,Novice,Lava Hot,j,A,Gb,1,3
199996,499996,1,0,0,F,Y,Green,Trapezoid,Lion,China,...,cfbd87ed0,eae3446d0,1,Contributor,Lava Hot,f,S,Ed,2,2
199997,499997,0,1,1,T,Y,Green,Trapezoid,Lion,Canada,...,1108bcd6c,33dd3cf4b,1,Novice,Boiling Hot,g,V,TR,3,1
199998,499998,1,0,0,T,Y,Blue,Star,Hamster,Costa Rica,...,606ac930b,d4cf587dd,2,Grandmaster,Boiling Hot,g,X,Ye,2,1


In [5]:
#원-핫 인코딩

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()                             # 원-핫 인코더 생성
all_data_encoded = encoder.fit_transform(all_data)    # 원-핫 인코딩 적용

In [6]:
# 데이터 나누기

num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
x_train = all_data_encoded[:num_train]    # 0~num_train ~ 1행
x_test = all_data_encoded                 # num_train~마지막행

y=train['target']

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

In [9]:
#모델 훈련

from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42) 
logistic_model.fit(x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [10]:
logistic_model.predict_proba(x_valid)

array([[0.23057724, 0.76942276],
       [0.91580546, 0.08419454],
       [0.84242849, 0.15757151],
       ...,
       [0.25273963, 0.74726037],
       [0.50728863, 0.49271137],
       [0.95770316, 0.04229684]])

In [11]:
logistic_model.predict(x_valid)

array([1, 0, 0, ..., 1, 0, 0])

In [14]:
# 검증 데이터를 활용한 타깃 예측
y_valid_preds = logistic_model.predict_proba(x_valid)[:,1]

In [15]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7976


In [17]:
# 예측 및 결과 제출
# 타깃값 1일 확률 예측

y_preds = logistic_model.predict_proba(x_test)[:,1]

In [19]:
# submission['target'] = y_preds
# submission.to_csv('submission.csv')